<a href="https://colab.research.google.com/github/francisluluaga/SEOPosts_Automation/blob/main/OpenAI_to_WP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install openai
!pip install requests
!pip install python-wordpress-xmlrpc
!pip install json
! pip install youtube_search
!pip install pillow
!pip install random
!pip install datetime

In [ ]:
import openai
import urllib
from wordpress_xmlrpc import Client, WordPressPost
import time
import requests
from wordpress_xmlrpc.methods.posts import NewPost
from wordpress_xmlrpc.methods.media import UploadFile
import json
from datetime import datetime, timedelta
import random


###############################################----- ACCESOS -----###############################################


#API OPENAI
openai.api_key = "API OPENAI"

#Accesos WordPress (Introducir dominio, sin http ni /)

DominioWordPress = "dominio.com"
Usuario = "Usuario"
Pass = "Contraseña"

#NOTA: Puede que tengas que poner la carpeta Uploads de tu WordPress con permisos 777.

###############################################----- FIN ACCESOS -----###############################################

#Instalamos clases Youtube
class YoutubeSearch:
      def __init__(self, search_terms: str, max_results=None):
          self.search_terms = search_terms
          self.max_results = max_results
          self.videos = self._search()
  
      def _search(self):
          encoded_search = urllib.parse.quote_plus(self.search_terms)
          BASE_URL = "https://youtube.com"
          url = f"{BASE_URL}/results?search_query={encoded_search}"
          response = requests.get(url).text
          while "ytInitialData" not in response:
              response = requests.get(url).text
          results = self._parse_html(response)
          if self.max_results is not None and len(results) > self.max_results:
              return results[: self.max_results]
          return results
  
      def _parse_html(self, response):
          results = []
          start = (
              response.index("ytInitialData")
              + len("ytInitialData")
              + 3
          )
          end = response.index("};", start) + 1
          json_str = response[start:end]
          data = json.loads(json_str)
  
          for contents in data["contents"]["twoColumnSearchResultsRenderer"]["primaryContents"]["sectionListRenderer"]["contents"]:
              for video in contents["itemSectionRenderer"]["contents"]:
                  res = {}
                  if "videoRenderer" in video.keys():
                      video_data = video.get("videoRenderer", {})
                      res["id"] = video_data.get("videoId", None)
                      res["thumbnails"] = [thumb.get("url", None) for thumb in video_data.get("thumbnail", {}).get("thumbnails", [{}]) ]
                      res["title"] = video_data.get("title", {}).get("runs", [[{}]])[0].get("text", None)
                      res["long_desc"] = video_data.get("descriptionSnippet", {}).get("runs", [{}])[0].get("text", None)
                      res["channel"] = video_data.get("longBylineText", {}).get("runs", [[{}]])[0].get("text", None)
                      res["duration"] = video_data.get("lengthText", {}).get("simpleText", 0)
                      res["views"] = video_data.get("viewCountText", {}).get("simpleText", 0)
                      res["publish_time"] = video_data.get("publishedTimeText", {}).get("simpleText", 0)
                      res["url_suffix"] = video_data.get("navigationEndpoint", {}).get("commandMetadata", {}).get("webCommandMetadata", {}).get("url", None)
                      #results.append(res)
  
              if results:
                  return results
          return results
  
      def to_dict(self, clear_cache=True):
          result = self.videos
          if clear_cache:
              self.videos = ""
          return result
  
      def to_json(self, clear_cache=True):
          result = json.dumps({"videos": self.videos})
          if clear_cache:
              self.videos = ""
          return result
  
from youtube_search import YoutubeSearch

      # Conectarse al sitio de WordPress
wp = Client('https://' + DominioWordPress + '/xmlrpc.php', Usuario, Pass)

      # Abrir el archivo txt con las palabras
with open("words.txt", "r", encoding="utf8") as f:
    words = f.readlines()

      # Número de palabras a generar
num_words = 1000

          # Generar texto para cada línea de palabras
for i, line in enumerate(words):
    
    try:      
              print(f"Procesando línea {i+1}: {line}")

              # Generar introducción
              intro_prompt = f"Escribe una introduccion para un articulo titulado {line}."
              intro_response = openai.Completion.create(engine="text-davinci-003", prompt=intro_prompt, temperature=0.7, max_tokens=500)
              intro = intro_response["choices"][0]["text"]
              time.sleep(2)

              # Generar el texto
              prompt = f"Escribe un artículo lo mas extenso posible sobre {line}. Debe tener como mínimo {num_words} palabras. Evita hacer una introducción. El texto debe estar en formato HTML incluyendo encabezados H2 sacados de las preguntas frecuentes de Google, resaltando en negrita lo más importante e incluye enlaces si es necesario. Comienza con una introducción en texto plano y luego añade los respectivos H2. En ningún caso añadadas H1. Piensa bien cada paso y si no estas seguro de algo no lo escribas."
              response = openai.Completion.create(engine="text-davinci-003", prompt=prompt, temperature=0.7, max_tokens=3000)
              text = response["choices"][0]["text"]
              time.sleep(2)

              # Generar final
              final_prompt = f"Escribe una conclusion a un articulo con titulo {line}. Resalta lo importante en negrita."
              final_response = openai.Completion.create(engine="text-davinci-003", prompt=final_prompt, temperature=0.5, max_tokens=500)
              final = final_response["choices"][0]["text"]
              time.sleep(2)

              # Generar el título
              title_prompt = f"Dime en texto plano y sin comas ni comillas un meta título posible y optimizado para SEO para la siguiente palabra clave: {line}:"
              title_response = openai.Completion.create(engine="text-davinci-003", prompt=title_prompt, temperature=0.5, max_tokens=120)
              title = title_response["choices"][0]["text"]
              time.sleep(2)

              # Generar la categoría
              cat_prompt = f"Categoriza el título de un artículo llamado {title} en una de las siguientes categorías: Tecnología, Entretenimiento, Negocios, Finanzas, Moda, Belleza, Ciencia, Salud, Viajes, Cocina, Educación, Jardinería, Deportes, Seguridad, Familia, Autoayuda, Comida, Artesanía, Arte, Literatura, Política, Ciencia Ficción, Historia, Fotografía, Juegos, Marketing, Redes Sociales, SEO, Productividad, Mascotas, Psicología, Sexualidad, Sociedad, Trabajo, Recetas Españolas, Yoga, Otros."
              cat_response = openai.Completion.create(engine="text-davinci-003", prompt=cat_prompt, temperature=0.5, max_tokens=50)
              cat = cat_response["choices"][0]["text"]

              # Obtener y agregar el video de YouTube
              jsonData = YoutubeSearch((line), max_results=1).to_json()
              jsonToPython = json.loads(jsonData)
              videosy = (jsonToPython["videos"])

              for entry in videosy:
                  urly = (entry['url_suffix'])

              string = (urly)
              characters = "/watch?v="

              inic = 9
              fina = 20

              original_string = (string)
              yvideo = urly[inic:fina]


              # Generar la imagen
              image_prompt = f"{line}" 
              data = """
              {
                  "model": "image-alpha-001",
                  "prompt": """ + json.dumps(image_prompt) + """,
                  "size":"256x256"
              }
              """
              image_response = requests.post("https://api.openai.com/v1/images/generations",
              data=data,
              headers={
                  "Content-Type": "application/json",
                  "Authorization": f"Bearer {openai.api_key}"
              }
          ).json()


          # Descargar la imagen
              image_url = image_response["data"][0]["url"]
              image_data = requests.get(image_url).content

          # Asignar el nombre del archivo como el título del artículo
              filename = title + ".png"
    
          # Subir la imagen a WordPress
              image_id = wp.call(UploadFile({'name': filename, 'bits': image_data}))
              image_url = image_id['url']

          # Genera un número aleatorio de días a publicar entre 0 y 20, 30, 40... 
              random_days = random.randint(0, 3)

          
          # Crea el post
              post = WordPressPost()
              future_date = datetime.now() + timedelta(days=random_days)
              post.date = future_date
              post.title = title
              post.terms_names = {
        'category': [cat]
      }
              post.content = intro + '<br><center><iframe width="560" height="315" src="https://www.youtube.com/embed/' + yvideo + '"></iframe></center><br>' + text + '<br><center><img src="' + image_url + '" alt="' + line + '"></center>' + final
              post.post_status = 'publish'
              wp.call(NewPost(post))
              time.sleep(10)
    except:
        print(f"Error en línea {i+1}: {line}")
        time.sleep(60)
        continue